# Overview
---
> In this Notebook, we will explore PyApacheAtlas package to work with Apache Atlas API in order to create custom Types, Entities and Relationships. 
All of that to make **Azure Analysis Services** assets available in Azure Purview.

**Scanning of Analysis Services** is the second most voted idea on Microsoft's Feedback forum (<a href="https://feedback.azure.com/forums/932437-azure-purview/suggestions/42183862-scanning-of-analysis-services-azure-analysis-serv"> Scanning AAS Idea</a>), but this feature currently is not in the product's roadmap.
Several companies have a large footprint of Analysis Services and they need to keep track of that.

This sample will help you to get started on create type structures, create entities and relationship to:
- Azure Analysis Services SERVER
- Azure Analysis Services MODELS
- Azure Analysis Services TABLES
- Azure Analysis Services COLUMNS

### Prerequisites

- [Python 3](https://www.python.org/downloads/)
- [Install Atlas Python client](https://github.com/wjohnson/pyapacheatlas)
<br>`python -m pip install pyapacheatlas`
- [Create and configure a Service Principal to use Purview APIs](https://docs.microsoft.com/en-us/azure/purview/tutorial-using-rest-apis)



In [ ]:
import json
import os
from pyapacheatlas.auth import ServicePrincipalAuthentication
from pyapacheatlas.core import PurviewClient
from pyapacheatlas.core import AtlasEntity, AtlasProcess
from pyapacheatlas.core import AtlasAttributeDef, EntityTypeDef, RelationshipTypeDef
from pyapacheatlas.core.util import GuidTracker
from pyapacheatlas.readers import ExcelReader
from pyapacheatlas.scaffolding import column_lineage_scaffold
from pyapacheatlas.readers import ExcelConfiguration, ExcelReader

#### Create a Purview Client Connection
> Provides connectivity to your Atlas / Azure Purview service.

In [ ]:
auth = ServicePrincipalAuthentication(
    tenant_id = "Your-Tenant-ID", # fill in
    client_id = "Your-Service-Principal-Client_ID", # fill in
    client_secret = "Your-Service-Principal-Secret" # fill in
)

# Create a client to connect to your service.
client = PurviewClient(
    account_name = "Your-Purview-Account-Name", # fill in
    authentication = auth
)

#### Create a custom entity type that represents the Azure Analysis Services Server
> What are the attributes that represents Analysis Services **Server**? <br>
- Azure Tier
- Subscription
- Resource Group
- Location
<br> You can have many others attribute that make sense for you, this is completelly customizabe.
<br> PS: superType is configured as "azure_resource" because this options is beeing based on Azure SQL Server superType. 
    

In [ ]:
custom_type = EntityTypeDef(
    name="azure_analysis_services_server",
    superTypes=["azure_resource"],
    description="azure_analysis_services_server",
    typeVersion="1.0",
)

#Attributes that represents the server
custom_type.addAttributeDef(
    AtlasAttributeDef(name="tier", typeName="string", isOptional=False),
    AtlasAttributeDef(name="subscription", typeName="string", isOptional=False),
    AtlasAttributeDef(name="resource-group", typeName="string", isOptional=False),
    AtlasAttributeDef(name="location", typeName="string", isOptional=False)
)

#Upload that custom type with the client
client.upload_typedefs(
    entityDefs=[custom_type],
    force_update=True
)

#### Create a custom entity type that represents the Azure Analysis Services Model
> What are the attributes that represents Analysis Services **Model**? <br>
- Database Name
- Compatibility Level
<br> You can have many others attribute that make sense for you, this is completelly customizabe.
<br> PS: superType is configured as "Asset" because this options is beeing based on Azure SQL Schema superType. 

In [ ]:
custom_type = EntityTypeDef(
    name="azure_analysis_services_model",
    superTypes=["Asset"],
    description="azure_analysis_services_model",
    typeVersion="1.0",
    serviceType="Azure Analysis Services Model",
)

#Attributes that represents the model
custom_type.addAttributeDef(
    AtlasAttributeDef(name="database-name", typeName="string", isOptional=False),
    AtlasAttributeDef(name="compatibility-level", typeName="int", isOptional=False)
)

#Upload that custom type with the client
client.upload_typedefs(
    entityDefs=[custom_type],
    force_update=True
)

#### Create a custom entity type that represents the Azure Analysis Services Model Table
> What are the attributes that represents Analysis Services Model **Table**? <br>
- Table Type 
- Query
<br> You can have many others attribute that make sense for you, this is completelly customizabe.
<br> PS: superType is configured as "DataSet" because this options is beeing based on Azure SQL Table superType. 

In [ ]:
custom_type = EntityTypeDef(
    name="azure_analysis_services_table",
    superTypes=["DataSet"],
    description="azure_analysis_services_table",
    typeVersion="1.0",
    serviceType="Azure Analysis Services Model",
)

#Attributes that represents the model table
custom_type.addAttributeDef(
    AtlasAttributeDef(name="tableType", typeName="string", isOptional=False),
    AtlasAttributeDef(name="query", typeName="string", isOptional=False)
)

client.upload_typedefs(
    entityDefs=[custom_type],
    force_update=True
)

#### Create a custom entity type that represents the Azure Analysis Services Model Table Column
> What are the attributes that represents Analysis Services Model Table **Column**? <br>
- Data Type
- isHidden
- isKey
- Column Type
<br> You can have many others attribute that make sense for you, this is completelly customizabe.
<br> PS: superType is configured as "DataSet" because this options is beeing based on Azure SQL Cloumn superType. 

In [ ]:
custom_type = EntityTypeDef(
    name="azure_analysis_services_column",
    superTypes=["DataSet"],
    description="azure_analysis_services_column",
    typeVersion="1.0",
    serviceType="Azure Analysis Services Model",
)

#Attributes that represents the model column
custom_type.addAttributeDef(
    AtlasAttributeDef(name="data_type", typeName="string", isOptional=False),
    AtlasAttributeDef(name="isHidden", typeName="string", isOptional=False),
    AtlasAttributeDef(name="isKey", typeName="string", isOptional=False),
    AtlasAttributeDef(name="columnType", typeName="string", isOptional=False)
)

client.upload_typedefs(
    entityDefs=[custom_type],
    force_update=True
)

#### Create a custom type that represents the Relationship between AAS Servers and Models

In [ ]:
rd = RelationshipTypeDef(
    name="azure_analysis_services_server_models",
    attributeDefs=[],
    relationshipCategory="COMPOSITION", # Means the child can't exist  without the parent
    endDef1={ # endDef1 decribes what the parent will have as an attribute
        "type":"azure_analysis_services_server", # Type of the parent
        "name":"models", # What the parent will have
        "isContainer": True,
        "cardinality":"SET", # This is related to the cardinality, in this case the parent Server will have a SET of Models.
        "isLegacyAttribute":False
    },
    endDef2={ # endDef2 decribes what the child will have as an attribute
        "type":"azure_analysis_services_model", # Type of the child
        "name":"server", # What the child will have
        "isContainer":False,
        "cardinality":"SINGLE",
        "isLegacyAttribute":False
    }
)
client.upload_typedefs(relationshipDefs=[rd])


#### Create a custom type that represents the Relationship between AAS Models and Tables

In [ ]:
rd = RelationshipTypeDef(
    name="azure_analysis_services_model_tables",
    attributeDefs=[],
    relationshipCategory="COMPOSITION",
    endDef1={
        "type":"azure_analysis_services_model",
        "name":"tables",
        "isContainer":True,
        "cardinality":"SET",
        "isLegacyAttribute":False
    },
    endDef2={
        "type":"azure_analysis_services_table",
        "name":"model",
        "isContainer":False,
        "cardinality":"SINGLE",
        "isLegacyAttribute":False
    }
)
client.upload_typedefs(relationshipDefs=[rd])

#### Create a custom type that represents the Relationship between AAS Tables and Columns

In [ ]:
rd = RelationshipTypeDef(
    name="azure_analysis_services_table_columns",
    attributeDefs=[],
    relationshipCategory="COMPOSITION",
    endDef1={
        "type":"azure_analysis_services_table",
        "name":"columns",
        "isContainer":True,
        "cardinality":"SET",
        "isLegacyAttribute":False
    },
    endDef2={
        "type":"azure_analysis_services_column",
        "name":"table",
        "isContainer":False,
        "cardinality":"SINGLE",
        "isLegacyAttribute":False
    }
)
client.upload_typedefs(relationshipDefs=[rd])

#### Create an Entity (AAS Server) based on that EntityTypeDef that was created before

In [ ]:
gt = GuidTracker() # Instantiate GuidTracker to manage guid creation
serverGuid = gt.get_guid()

#Create a new (AAS Server) entity
ae = AtlasEntity(
    name="AASHOMSERVER",
    typeName="azure_analysis_services_server",
    qualified_name="asazure://exsaaseastus2.asazure.windows.net/aashomserver",# AAS Server name obtained on Azure Portal
    guid=serverGuid,
    attributes={"tier":"D5", "subscription":"GProd", "resource-group":"USAZU1VRES302", "location":"EAST US 2"}
)
# Upload the new entity
uploadEntity = client.upload_entities(batch=[ae])

#### Create an Entity (AAS Model) based on that EntityTypeDef that was created before

In [ ]:
modelGuid1 = gt.get_guid()

#Create a new (AAS Model) entity
ae = AtlasEntity(
    name="Model1",
    typeName="azure_analysis_services_model",
    qualified_name="asazure://exsaaseastus2.asazure.windows.net/aashomserver/model1",
    guid=modelGuid1,
    attributes={"database-name":"AASDB1", "compatibility-level":1100}
)
# Upload the new entity
uploadEntity = client.upload_entities(batch=[ae])

#### Create an Entity (AAS Model) based on that EntityTypeDef that was created before

In [ ]:
modelGuid2 = gt.get_guid()

#Create a new (AAS Model) entity
ae = AtlasEntity(
    name="Model2",
    typeName="azure_analysis_services_model",
    qualified_name="asazure://exsaaseastus2.asazure.windows.net/aashomserver/model2",
    guid=modelGuid2,
    attributes={"database-name":"AASDB1", "compatibility-level":1100}
)
# Upload the new entity
uploadEntity = client.upload_entities(batch=[ae])

#### Create an Entity (AAS Table) based on that EntityTypeDef that was created before

In [ ]:
tableGuid = gt.get_guid()

#Create a new (AAS Table) entity
ae = AtlasEntity(
    name="Table1",
    typeName="azure_analysis_services_table",
    qualified_name="asazure://exsaaseastus2.asazure.windows.net/aashomserver/model1/table1",
    guid=tableGuid,
    attributes={"tableType":"Data", "query": ""}
)
# Upload the new entity
uploadEntity = client.upload_entities(batch=[ae])

#### Create an Entity (AAS Column) based on that EntityTypeDef that was created before

In [ ]:
columnGuid = gt.get_guid()

#Create a new (AAS Column) entity
ae = AtlasEntity(
    name="Column1",
    description="Column description",
    typeName="azure_analysis_services_column",
    qualified_name="asazure://exsaaseastus2.asazure.windows.net/aashomserver/model1/table1#column1",
    guid=columnGuid,
    attributes={"data_type":"String", "isHidden": "FALSE", "isKey":"FALSE", "columnType":"Data"}
)
# Upload the new entity
uploadEntity = client.upload_entities(batch=[ae])

> This is an exemple on how may be appearing on Azure Purview at this point:
<img src="./img/searchResults.png" width="700" height="700">

---

### Now create the relationship between the entities, using the guid reference for the entity created before
#### Create the relationship between the server (AASHOMSERVER) and model (model1)

In [ ]:
relationship = {
    "typeName": "azure_analysis_services_server_models",
    "attributes": {},
    "guid": -100,
    "provenanceType": 0,
    "end1": {
        "guid": serverGuid,
        "typeName": "azure_analysis_services_server",
        "uniqueAttributes": {"qualifiedName": "asazure://exsaaseastus2.asazure.windows.net/aashomserver"}

    },
    "end2": {
        "guid": modelGuid1,
        "typeName": "azure_analysis_services_model",
        "uniqueAttributes": {"qualifiedName": "asazure://exsaaseastus2.asazure.windows.net/aashomserver/model1"}
    }
}
# Upload the new relationship
client.upload_relationship(relationship)

#### Create the relationship between the server (AASHOMSERVER) and model (model2)

In [ ]:
relationship = {
    "typeName": "azure_analysis_services_server_models",
    "attributes": {},
    "guid": -100,
    "provenanceType": 0,
    "end1": {
        "guid": serverGuid,
        "typeName": "azure_analysis_services_server",
        "uniqueAttributes": {"qualifiedName": "asazure://exsaaseastus2.asazure.windows.net/aashomserver"}

    },
    "end2": {
        "guid": modelGuid2,
        "typeName": "azure_analysis_services_model",
        "uniqueAttributes": {"qualifiedName": "asazure://exsaaseastus2.asazure.windows.net/aashomserver/model2"}
    }
}
# Upload the new relationship
client.upload_relationship(relationship)

> This is an exemple on how may be appearing on Azure Purview at this point:
<img src="./img/serverRelationWithModels.png" width="800" height="800">

---

#### Create the relationship between the model (model1) and table (table1)

In [ ]:
relationship = {
    "typeName": "azure_analysis_services_model_tables",
    "attributes": {},
    "guid": -100,
    "provenanceType": 0,
    "end1": {
        "guid": modelGuid1,
        "typeName": "azure_analysis_services_model",
        "uniqueAttributes": {"qualifiedName": "asazure://exsaaseastus2.asazure.windows.net/aashomserver/model1"}
    },
    "end2": {
        "guid": tableGuid,
        "typeName": "azure_analysis_services_table",
        "uniqueAttributes": {"qualifiedName": "asazure://exsaaseastus2.asazure.windows.net/aashomserver/model1/table1"}
    }
}
# Upload the new relationship
client.upload_relationship(relationship)

#### Create the relationship between the table (table1) and column (column1)

In [ ]:
relationship = {
    "typeName": "azure_analysis_services_table_columns",
    "attributes": {},
    "guid": -100,
    "provenanceType": 0,
    "end1": {
        "guid": tableGuid,
        "typeName": "azure_analysis_services_table",
        "uniqueAttributes": {"qualifiedName": "asazure://exsaaseastus2.asazure.windows.net/aashomserver/model1/table1"}
    },
    "end2": {
        "guid": columnGuid,
        "typeName": "azure_analysis_services_column",
        "uniqueAttributes": {"qualifiedName": "asazure://exsaaseastus2.asazure.windows.net/aashomserver/model1/table1#column1"}
    }
}
# Upload the new relationship
client.upload_relationship(relationship)

> This is an exemple on how may be appearing on Azure Purview at this point:
<img src="./img/tableRelationWithColumns.png" width="800" height="800">

---

### Clean Up

In [ ]:
# Deletes all entities and custom types
client.delete_entity(guid=modelGuid1)
client.delete_entity(guid=modelGuid2)
client.delete_entity(guid=serverGuid)
client.delete_entity(guid=tableGuid)
client.delete_type(name="azure_analysis_services_server_models")
client.delete_type(name="azure_analysis_services_model_tables")
client.delete_type(name="azure_analysis_services_table_columns")
client.delete_type(name="azure_analysis_services_server")
client.delete_type(name="azure_analysis_services_model")
client.delete_type(name="azure_analysis_services_table")
client.delete_type(name="azure_analysis_services_column")

### Conclusions
> This is a sample to help you to represents Azure Analysis Services on Azure Purviiew, creating and uploading the assets in a manual manner. <br> The point here is to give an overall ideia on how to configure everything and see how it works. <br><br>
> For operationalize and make this process more automatic, you may get the metadata from AAS through API (<a href="https://docs.microsoft.com/en-us/rest/api/analysisservices/">AAS REST API</a>) and upload everything using a Excel template file to do so (<a href="https://github.com/wjohnson/pyapacheatlas/tree/master/samples/excel">Excel Sample</a>).